In [1]:
# importing dependencies

import pandas as pd
import re
import spacy
import numpy as np

In [ ]:
# url = 'https://drive.google.com/uc?id=1PYaJj1JKVpS9PD6spplswLb0hegiSu6P'

# output = '/home/jupyter/Ensemble/X_train_update.csv'
# gdown.download(url, output, quiet=False)

In [2]:
# Import training and testing data 
X_train = pd.read_csv('/home/jupyter/Ensemble/X_train_update.csv', index_col=0)

X_test = pd.read_csv('/home/jupyter/Ensemble/X_test_update.csv', index_col=0)


In [3]:
y_train = pd.read_csv('/home/jupyter/Ensemble/Y_train_CVw08PX.csv', index_col=0)

In [4]:
y_train.shape

(84916, 1)

In [ ]:
# visualizing class distribution 

# Number of images per class label
y_train.prdtypecode.value_counts()

In [ ]:
#visualize the datframes

X_train.head()


In [ ]:
#visualize the datframes

X_test.head()

In [ ]:
#visualize the datframes

y_train.head()

In [ ]:
# dropping the irrelevant columns from the data frames

del X_train['description'], X_train['productid'], X_train['imageid']

del X_test['description'], X_test['productid'], X_test['imageid']

#renaming the column in y train 
y_train = y_train.rename(columns={"prdtypecode": "label"})

Now for all designation column in, we :

1. remove the accents - done
2. make to lowercase - done
3. remove the special char like degree or phi - done
#To be discussed - may we should remove the numbers as well
4. Tokenize - Done
5. Handling Punctuations
4. Stop work removal  - 

In [ ]:
# Function to normalize
def normalize_accent(string):
    string = string.replace('á', 'a')
    string = string.replace('â', 'a')

    string = string.replace('é', 'e')
    string = string.replace('è', 'e')
    string = string.replace('ê', 'e')
    string = string.replace('ë', 'e')

    string = string.replace('î', 'i')
    string = string.replace('ï', 'i')

    string = string.replace('ö', 'o')
    string = string.replace('ô', 'o')
    string = string.replace('ò', 'o')
    string = string.replace('ó', 'o')

    string = string.replace('ù', 'u')
    string = string.replace('û', 'u')
    string = string.replace('ü', 'u')

    string = string.replace('ç', 'c')

    
    return string

In [ ]:
# function to tokenize
def raw_to_tokens(raw_string, spacy_nlp):
    # Write code for lower-casing
    string = raw_string.lower()
    
    # Write code to normalize the accents
    string = normalize_accent(string)
        
    # Write code to tokenize
    spacy_tokens = spacy_nlp(string)
        
    # Write code to remove punctuation tokens and create string tokens
    string_tokens = [token.orth_ for token in spacy_tokens if not token.is_punct if not token.is_stop]
    
    # Write code to join the tokens back into a single string
    clean_string = " ".join(string_tokens)
    
    return clean_string

In [ ]:
# # remove the accents
# X_train['designation_deaccent'] = X_train['designation'].apply(lambda x: normalize_accent(x))
# X_test['designation_deaccent'] = X_test['designation'].apply(lambda x: normalize_accent(x))

In [ ]:
# # make to lowercase
# X_train['designation_lower'] = X_train['designation_deaccent'].apply(lambda x: x.lower())
# X_test['designation_lower'] = X_test['designation_deaccent'].apply(lambda x: x.lower())

In [ ]:
# remove the special char like degree or phi 
X_train['designation_sp'] = X_train['designation'].apply(lambda x: re.sub('\W+'," ", x ))
X_test['designation_sp'] = X_test['designation'].apply(lambda x: re.sub('\W+'," ", x ))

In [ ]:
# reomve numbers
X_train['designation_number'] = X_train['designation_sp'].str.replace('\d+', '')
X_test['designation_number'] = X_test['designation_sp'].str.replace('\d+', '')

In [ ]:
# import and load spacy and download the french package

# !python -m spacy download fr


# Load spaCy for french
spacy_nlp = spacy.load('fr')

In [ ]:
# tokenize the train file
X_train['designation_token'] = X_train['designation_number'].apply(lambda x: raw_to_tokens(x, spacy_nlp))

In [ ]:
# tokenize the test file
X_test['designation_token'] = X_test['designation_number'].apply(lambda x: raw_to_tokens(x, spacy_nlp))

In [ ]:
# save to a csv for future use
X_train.to_csv('/home/jupyter/Ensemble/X_train_mid1.csv', encoding='utf-8', index=False)

In [ ]:
# save to a csv for future use
X_test.to_csv('/home/jupyter/Ensemble/X_test_mid1.csv', encoding='utf-8', index=False)

## Since the above step takes a lot of time to run, the dataframe is saved as a csv after it and then, the csv file can be loaded as it is.

In [5]:
X_train = pd.read_csv('/home/jupyter/Ensemble/X_train_mid1.csv', encoding='utf-8')

In [6]:
X_test = pd.read_csv('/home/jupyter/Ensemble/X_test_mid1.csv', encoding='utf-8')

In [1]:
import pandas as pd
y_train = pd.read_csv('/home/jupyter/Ensemble/Y_train_CVw08PX.csv', index_col=0)

In [7]:
X_train.shape

(84916, 4)

In [8]:
desc_list_train = X_train['designation']

In [9]:
desc_list_test = X_test['designation']

In [ ]:
frames = [desc_list_train, desc_list_test]
consolidated_desc = pd.concat(frames)

In [ ]:
y_train.shape

In [ ]:
# Write code to import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Write code to create a TfidfVectorizer object
tfidf = TfidfVectorizer()

# Write code to vectorize the sample text
X_tfidf_sample = tfidf.fit_transform(consolidated_desc.astype('U'))

print("Shape of the TF-IDF Matrix:")
print(X_tfidf_sample.shape)


In [ ]:
# optinal step to print details about the matrix
print("TF-IDF Matrix:")
print(X_tfidf_sample.todense())
print(tfidf.get_feature_names())

# We can now save the matrix and import it for our use later


In [10]:
from scipy import sparse

sparse.save_npz("/home/jupyter/Ensemble/X_consolidated_without_numbers.npz", X_tfidf_sample)

NameError: name 'X_tfidf_sample' is not defined

In [11]:
#Loading the matrix
from scipy import sparse
X_tfidf_sample = sparse.load_npz("/home/jupyter/Ensemble/X_consolidated_without_numbers.npz")

In [12]:
print("Shape of the TF-IDF Matrix:")
print(X_tfidf_sample.shape)

Shape of the TF-IDF Matrix:
(98728, 64055)


In [13]:
X_train = X_tfidf_sample[:84916, : ]
X_test = X_tfidf_sample[84916:, : ]

In [14]:
X_test.shape

(13812, 64055)

In [15]:
X_train.shape

(84916, 64055)

In [16]:
y_train.shape

(84916, 1)

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier  # use decision tree as base estimator
bagging = BaggingClassifier(None, 20, 0.5, 0.5)

bagging.fit(X_train, y_train.values.ravel())

In [ ]:
without_numbers = pd.DataFrame(bagging.predict(X_test))

In [ ]:
without_numbers.to_csv('without_numbers_and_SC.csv')

## Now I will try random forest and boosting to get the best accuracy resluts and hence extrapolate the one on the entire dataset

# 1. Random forest 

In [ ]:
y_train.shape

In [ ]:
# before we deine our new training data, consisting of just 25% of the total data

# X_train_grid = X_tfidf_sample[:21234, : ]
# X_test_grid = X_tfidf_sample[84916:, : ]
# y_train_grid = y_train.iloc[:21234, : ]

In [9]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(50, 150, num = 10)]

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 200, num = 10)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [50, 61, 72, 83, 94, 105, 116, 127, 138, 150], 'max_depth': [10, 31, 52, 73, 94, 115, 136, 157, 178, 200, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


In [10]:
from sklearn.ensemble import RandomForestClassifier

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,
                               n_iter = 20, cv = 3, verbose=1, random_state=42, scoring = 'f1_weighted')

# Fit the random search model
rf_random.fit(X_train, y_train.values.ravel())

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed: 95.3min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [11]:
rf_random.best_params_

{'n_estimators': 50,
 'min_samples_split': 10,
 'min_samples_leaf': 1,
 'max_depth': 178,
 'bootstrap': False}

In [12]:
rf_random.cv_results_

{'mean_fit_time': array([140.40719144,   9.3549188 ,  30.88645101,  89.83845814,
        124.89654175, 132.02254089, 112.65080953,  59.61499405,
        126.19765369,  11.68609269,  43.94501336,   9.87197081,
        208.77011077,  47.4842279 , 136.63955665, 217.78881844,
        116.13031642,  53.44487143, 153.79977632,  59.82852991]),
 'std_fit_time': array([0.74994576, 0.02510521, 0.09757134, 0.14512144, 0.68574227,
        0.77118391, 1.40720057, 0.20873075, 0.57440172, 0.04582493,
        0.0431229 , 0.04347796, 0.44130817, 0.10016098, 0.60393185,
        1.26997071, 0.22188391, 0.12085492, 1.1763318 , 0.2450794 ]),
 'mean_score_time': array([1.40923389, 0.49251469, 0.56308293, 0.80073396, 0.94220487,
        1.15408246, 1.27679094, 0.97044357, 1.11701147, 0.62109629,
        0.69996262, 0.36167288, 1.79140059, 0.96000854, 1.19544021,
        1.59905386, 1.25843318, 0.80574711, 2.2896688 , 0.89618556]),
 'std_score_time': array([0.00714354, 0.00659295, 0.00529637, 0.00347318, 0.01

In [14]:
rand_boosting = pd.DataFrame(rf_random.predict(X_test))

In [15]:
rand_boosting.to_csv('rand_boosting.csv')

## The model based on random search of hyper parameters

In [ ]:
from sklearn.model_selection import cross_val_score

clf_rand = RandomForestClassifier(n_estimators = 167, min_samples_split = 5, min_samples_leaf = 1, max_depth = None ,max_features = 'auto',
 bootstrap = False)
cross_val_score(clf_rand, X_train,y_train.values.ravel(), cv=3, scoring = 'accuracy')

## The below is the base model 

In [ ]:
clf2 = RandomForestClassifier(n_estimators= 100)
cross_val_score(clf2, X_train,y_train.values.ravel(), cv=3, scoring = 'accuracy')

# Gradient Boosting Classifier

In [17]:
from sklearn import ensemble

# the classifier with the hyper parameters tuned

In [25]:
import numpy as np

# n_estimators =  10, max_leaf_nodes = 4, max_depth = None, random_state = 2,
#                    min_samples_split =  5

from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(5, 150, num = 10)]

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 100, num = 10)]
max_depth.append(None)

# Minimum number of samples required to split a node
max_leaf_nodes = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 5, 10]


# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'max_leaf_nodes': max_leaf_nodes,
               'min_samples_leaf': min_samples_leaf}
print(random_grid)

{'n_estimators': [5, 21, 37, 53, 69, 85, 101, 117, 133, 150], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None], 'max_leaf_nodes': [2, 5, 10], 'min_samples_leaf': [1, 5, 10]}


In [32]:
from sklearn.ensemble import RandomForestClassifier

# Use the random grid to search for best hyperparameters
# First create the base model to tune
gb = ensemble.GradientBoostingClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
gb_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,
                               n_iter = 20, cv = 3, verbose=1, random_state=42,n_jobs = -1, scoring = 'f1_weighted')

# Fit the random search model
gb_random.fit(X_train, y_train.values.ravel())

Fitting 3 folds for each of 20 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed:   38.1s finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [27]:
gb_random.best_params_

{'n_estimators': 21,
 'min_samples_leaf': 1,
 'max_leaf_nodes': 10,
 'max_depth': 40}

In [28]:
gb_random.cv_results_

{'mean_fit_time': array([ 4.4350547 ,  3.29407748,  1.76290504, 12.42242233, 20.6821808 ,
        13.28680436,  0.22645156,  4.55982304, 12.39670126,  8.35699375,
        20.31031036, 13.13265204,  0.79666193,  3.9857192 ,  9.66266418,
         5.88176799,  4.39024623,  3.0557371 ,  9.32074372,  6.14844076]),
 'std_fit_time': array([0.02349409, 0.01928056, 0.01209299, 0.05304162, 0.02477188,
        0.08143524, 0.0030735 , 0.01699617, 0.033125  , 0.01790237,
        1.08735472, 0.05722286, 0.01490909, 0.02261185, 0.03392642,
        0.0175904 , 0.04200916, 0.04692923, 0.02411914, 0.47221721]),
 'mean_score_time': array([0.3949484 , 0.17895921, 0.16974338, 0.95647033, 0.96400277,
        0.55267946, 0.06304312, 0.73580678, 0.93178217, 0.65168659,
        0.70117664, 0.57703455, 0.06693737, 0.7242581 , 0.79485337,
        0.97647039, 0.35976839, 0.2545499 , 0.45048936, 0.52248502]),
 'std_score_time': array([0.02771499, 0.00058732, 0.00072194, 0.01210556, 0.04219614,
        0.00685463, 

In [19]:
from sklearn.model_selection import cross_val_score

In [20]:
# gb_random.fit(X_train,y_train.values.ravel())

clf2 = ensemble.GradientBoostingClassifier(n_estimators= 50)

cross_val_score(clf2, X_train,y_train.values.ravel(), cv=3, scoring = 'f1_weighted')

array([0.68661185, 0.68093614, 0.68303433])

In [ ]:
gb_random = pd.DataFrame(gb_random.predict(X_test))

In [30]:
gb_random.to_csv('gb_random.csv')

# output from the matrix creted by random search

In [35]:
from sklearn.model_selection import cross_val_score

clf2 = ensemble.GradientBoostingClassifier(n_estimators= 20,
 min_samples_leaf = 5,
 max_leaf_nodes =2,
 max_depth = 20)

clf2.fit(X_train,y_train.values.ravel())

cross_val_score(clf, X_train,y_train.values.ravel(), cv=3, scoring = 'f1_weighted')

KeyboardInterrupt: 

In [ ]:
rand_boosting = pd.DataFrame(clf2.predict(X_test))

In [ ]:
rand_boosting.to_csv('rand_boosting.csv')

# this is the standard classifier

In [ ]:
import sklearn

In [ ]:
from sklearn.model_selection import cross_val_score

clf = ensemble.GradientBoostingClassifier(n_estimators =  50)
clf.fit(X_train,y_train.values.ravel())

#cross_val_score(clf, X_train,y_train.values.ravel(), cv=3, scoring = 'f1_weighted')

In [ ]:
std_boosting = pd.DataFrame(cross_val_score.predict(X_test))

In [ ]:
std_boosting.to_csv('std_boosting.csv')